<a href="https://colab.research.google.com/github/Albategnius/Python/blob/master/LLM_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@raufpokemon00/building-a-large-language-model-llm-from-scratch-61fed0570ea5#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjA3YjgwYTM2NTQyODUyNWY4YmY3Y2QwODQ2ZDc0YThlZTRlZjM2MjUiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMDg0MDg4MTA4OTMzMTY4MTQ0NjgiLCJlbWFpbCI6ImhhcmRpcm9iYnlAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTc0NjQ4NjQ1NSwibmFtZSI6IlJvYmJ5IEhhcmRpIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0s5Wk96TFJnbmtYNFZvWWR3Y1U2THlYX3MzM1g5eEpaTi1sUXFTbFloYzFaZDR5Y0ZvPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6IlJvYmJ5IiwiZmFtaWx5X25hbWUiOiJIYXJkaSIsImlhdCI6MTc0NjQ4Njc1NSwiZXhwIjoxNzQ2NDkwMzU1LCJqdGkiOiIyOGY0MzI0MjViZTE3ODlkZTFiNDVmN2MwN2MwMGExMjhkYjkyYTRhIn0.iuKE2lPMv7KZmgqDF6YXKDl334mJJD9eMlvI-Pbv35YfXnTsyGNBywggr6_jSlFpPJUwQDn8PQVRUdKRcC_5nrB_HyuejXxWqxzKQg4aqGLNHQO6agDDoM-2XBEzYNVHjY2uqVL1YN1ULw-TGvyqH0hJnpT254C-E6Mher60FVoqwQZVybfA6zlRCRaFG3YsIU_xJlmnoN9njCDgmXZJdCC9S3C8d_okyoIsT_2tWWmGG02WzMBkFZLA_PjzhI9IH3BgqoH-1WheJGoO_zm3YrTRdtXz0vU61WS1J20sHPzY4Bd9cD85VglOw2_YthZxHrR-L8bGhpuPdjmLonIbKg

In [2]:
# Scrapping

import requests
from bs4 import BeautifulSoup

# Step 1: Specify the URL
url = "https://wikipedia.com"

# Step 2: Send a GET request to the website
response = requests.get(url)

# Step 3: Parse the website content
soup = BeautifulSoup(response.text, "html.parser")

# Step 4: Extract all text from the page
text_data = soup.get_text()

# Step 5: Print the first 500 characters of the text
print(text_data[:500])

In [ ]:
# data processing

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Sample text
text = "My name is Abdul Rauf jatoi"

# Step 1: Tokenize text into words
tokens = word_tokenize(text)

# Step 2: Convert to lowercase and remove non-alphanumeric tokens
tokens = [word.lower() for word in tokens if word.isalnum()]

# Step 3: Remove stop words
filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]

# Display the preprocessed tokens
print(filtered_tokens)

# NLTK is great for beginners; spaCy is better for more advanced tasks.

In [ ]:
# Model Architecture

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention

# Define a single transformer block
def transformer_block(input, num_heads, key_dim):
    # Step 1: Multi-head attention
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(input, input)
    # Step 2: Add & Normalize
    attention = LayerNormalization()(attention + input)
    # Step 3: Feedforward network
    dense = Dense(128, activation='relu')(attention)
    # Step 4: Add & Normalize
    output = LayerNormalization()(dense + attention)
    return output

# Input layer
input_layer = Input(shape=(None, 128))  # Sequence length is variable, feature size is 128
# Transformer block
transformer_output = transformer_block(input_layer, num_heads=8, key_dim=64)
# Model definition
model = tf.keras.Model(inputs=input_layer, outputs=transformer_output)

# Model summary
model.summary()

In [ ]:
# Fine Tune GPT

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import torch

# Step 1: Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Step 2: Prepare the dataset (replace `my_dataset` with your dataset)
# Ensure `my_dataset` is a PyTorch dataset with tokenized input
# Example tokenization
texts = ["Hello, how are you?", "Fine-tuning is fun!"]
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
labels = encodings.input_ids.clone()

my_dataset = torch.utils.data.TensorDataset(encodings.input_ids, labels)

# Step 3: Define training arguments
train_args = TrainingArguments(
    output_dir='./results',        # Directory to save the model
    per_device_train_batch_size=4, # Batch size
    num_train_epochs=1,            # Number of epochs
    save_steps=10_000,             # Steps to save checkpoints
    save_total_limit=2,            # Maximum number of saved checkpoints
    logging_dir='./logs',          # Directory for logs
    logging_steps=500,             # Log every 500 steps
)

# Step 4: Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=dataset,
)

trainer.train()

In [ ]:
# Deployment

from flask import Flask, request, jsonify
from transformers import pipeline

# Initialize Flask app and model pipeline
app = Flask(__name__)
model = pipeline("text-generation", model="gpt2")

# Define API endpoint
@app.route("/generate", methods=["POST"])
def generate():
    # Get input text from the request
    input_text = request.json["input"]
    # Generate text using the model
    output = model(input_text, max_length=50)
    # Return the generated text as a JSON response
    return jsonify(output)

# Run the app
if __name__ == "__main__":
    app.run(debug=True)